In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
dx_dy = 0.0333958286584664
dt = 0.025

In [ ]:
# Define the function to scale the pulse data
def scale_pulse(image, pulse, dx_dy, dt):
    image_energy = image.sum() * dx_dy
    pulse_energy = pulse.sum() * dt
    K = image_energy / pulse_energy
    scaled_pulse = K * pulse
    return scaled_pulse


def load_data(input_dir, output_dir):
    # Get sorted lists of file names
    input_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.csv')])
    output_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.csv')])

    # Print mappings of input files to output files
    for input_file, output_file in zip(input_files, output_files):
        print(f"Mapping: {input_file} -> {output_file}")  # Print the mapping

        # Load input and output files without scaling
        input_data = pd.read_csv(os.path.join(input_dir, input_file), header=None).values.flatten()
        output_data = pd.read_csv(os.path.join(output_dir, output_file), header=None).values.flatten()

        input_train_data_list.append(input_data)
        output_train_data_list.append(output_data)

    return np.array(input_train_data_list), np.array(output_train_data_list)




In [ ]:
dx_dy = 0.0333958286584664
dt = 0.025

i_range = [i for i in range(20, 54)]
j_range = range(1, 41)

base_directory = '/content/drive/MyDrive/Data_LLE/Input_Pulse'
scaled_pulse_base_directory = '/content/drive/MyDrive/Scaled_Input_Pulse'
image_base_directory = '/content/drive/MyDrive/Data_LLE/Refined Data Set'

for i in i_range:
    for j in j_range:

        input_pulse_file = os.path.join(base_directory, f'Run_{i}_BL4_s39708_InjPulsePower_InjEnergyFactor_{j}.csv')
        scaled_pulse_base = os.path.join(scaled_pulse_base_directory, f'Run_{i}_BL4_s39708_InjPulsePower_InjEnergyFactor_{j}.csv')
        input_image_file = os.path.join(image_base_directory, f'Run {i} - BL4, s39708', f'Run_{i}_BL4_s39708_Inj_256x256_InjEnergyFactor_{j}.csv')


        if os.path.isfile(input_pulse_file) and os.path.isfile(input_image_file):

            image_data = pd.read_csv(input_image_file, header=None).values


            input_pulse = pd.read_csv(input_pulse_file, header=None).iloc[0].values
            scaled_input_pulse = scale_pulse(image_data, input_pulse)
            scaled_input_pulse = scaled_input_pulse.transpose()

            # Convert to DataFrame and save to CSV
            pd.DataFrame(scaled_input_pulse).to_csv(scaled_pulse_base, header=None, index=False)

        else:
            print(f"Files not found for i={i}, j={j}: Skipping.")

print("Data loading complete.")

In [ ]:
import os
import shutil
output_train_dir = '/content/drive/MyDrive/Scaled_Input_Pulse'
output_test_dir = '/content/drive/MyDrive/Scaled_Test/Output'
i_range = [22, 26, 30, 34, 48]
j_range = range(1, 41)
# Move files
for i in i_range:""
    for j in j_range:
        # Construct the file name
        file_name = f'Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.csv'
        source_path = os.path.join(output_train_dir, file_name)
        destination_path = os.path.join(output_test_dir, file_name)

        # Check if the file exists before moving
        if os.path.exists(source_path):
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} -> {destination_path}")
        else:
            print(f"File not found: {source_path}")

In [ ]:
# Lists to hold each loaded sample
input_train_data_list = []
output_train_data_list = []

def load_data(input_dir, output_dir):
    # Get sorted lists of file names
    input_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.csv')])
    output_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.csv')])

    for input_file, output_file in zip(input_files, output_files):
        # Load input and output files without scaling
        input_data = pd.read_csv(os.path.join(input_dir, input_file), header=None).iloc[:, 59:59+576].values.flatten()
        output_data = pd.read_csv(os.path.join(output_dir, output_file), header=None).iloc[:, 59:59+576].values.flatten()
        print(f"{input_file}: {input_data.shape}")
        print(f"{output_file}: {output_data.shape}")
        input_train_data_list.append(input_data)
        output_train_data_list.append(output_data)

    return np.array(input_train_data_list), np.array(output_train_data_list)

# Specify directories
input_train_dir = '/content/drive/MyDrive/Scaled_Input_Pulse'
output_train_dir = '/content/drive/MyDrive/Data_LLE/Output_Pulse'

# Load the data
X_train, y_train = load_data(input_train_dir, output_train_dir)

# Reshape X and y for LSTM compatibility (samples, time steps, features)
X_train = X_train.reshape(len(X_train), 576, 1)
y_train = y_train.reshape(len(y_train), 576, 1)

print(f"Shape of X (input): {X_train.shape}")
print(f"Shape of y (output): {y_train.shape}")

In [ ]:
# Lists to hold each loaded sample
input_test_data_list = []
output_test_data_list = []

def load_data(input_dir, output_dir):
    # Get sorted lists of file names
    input_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.csv')])
    output_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.csv')])

    for input_file, output_file in zip(input_files, output_files):
        # Load input and output files without scaling
        input_data = pd.read_csv(os.path.join(input_dir, input_file), header=None).iloc[:, 59:59+576].values.flatten()
        output_data = pd.read_csv(os.path.join(output_dir, output_file), header=None).iloc[:, 59:59+576].values.flatten()

        input_test_data_list.append(input_data)
        output_test_data_list.append(output_data)

    return np.array(input_test_data_list), np.array(output_test_data_list)

# Specify directories
input_test_dir = '/content/drive/MyDrive/Scaled_Test/Input'
output_test_dir = '/content/drive/MyDrive/Scaled_Test/Output'

# Load the data
X_test, y_test = load_data(input_test_dir, output_test_dir)

# Reshape X and y for LSTM compatibility (samples, time steps, features)
X_test = X_test.reshape(len(X_test), 576, 1)
y_test = y_test.reshape(len(y_test), 576, 1)

print(f"Shape of X (input): {X_test.shape}")
print(f"Shape of y (output): {y_test.shape}")

In [ ]:
input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

In [ ]:
# Reshape data to 2D for scaling, then reshape back
X_train_reshaped = X_train.reshape(-1, 2)  # Combine samples and time steps for scaling
X_train_normalized = input_scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)  # Reshape back to original shape

y_train_reshaped = y_train.reshape(-1, 1)  # Output is 1D per sample, so reshape to 2D for scaling
y_train_normalized = output_scaler.fit_transform(y_train_reshaped).reshape(y_train.shape)

In [ ]:
# Reshape data to 2D for scaling, then reshape back
X_test_reshaped = X_test.reshape(-1, 2)  # Combine samples and time steps for scaling
X_test_normalized = input_scaler.fit_transform(X_test_reshaped).reshape(X_test.shape)  # Reshape back to original shape

y_test_reshaped = y_test.reshape(-1, 1)  # Output is 1D per sample, so reshape to 2D for scaling
y_test_normalized = output_scaler.fit_transform(y_test_reshaped).reshape(y_test.shape)

In [ ]:
# Split data
X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train_normalized, y_train_normalized, test_size=0.15, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Custom correlation loss function
def correlation_loss(y_true, y_pred):
    # Calculate the mean of the true and predicted values
    y_true_mean = tf.reduce_mean(y_true)
    y_pred_mean = tf.reduce_mean(y_pred)

    # Calculate the covariance between the true and predicted values
    cov = tf.reduce_mean((y_true - y_true_mean) * (y_pred - y_pred_mean))

    # Calculate the standard deviations of the true and predicted values
    y_true_std = tf.math.reduce_std(y_true)
    y_pred_std = tf.math.reduce_std(y_pred)

    # Calculate the correlation coefficient
    correlation = cov / (y_true_std * y_pred_std + 1e-6)  # Adding epsilon to avoid division by zero

    # Return the negative correlation as the loss (since we want to maximize correlation)
    return correlation

def combined_loss(y_true, y_pred, alpha=0.5, beta=0.1):
    # L1 loss (Mean Absolute Error)
    l1_loss = tf.reduce_mean(tf.abs(y_true - y_pred))

    # Calculate the correlation coefficient between y_true and y_pred
    corr_loss = correlation_loss(y_true, y_pred)

    penalty = tf.where(corr_loss < 0, tf.abs(corr_loss), 0)

    return alpha * l1_loss + (1 - alpha) * (1 - corr_loss) + beta * penalty




In [ ]:
# Define the model
model = tf.keras.Sequential([

    # Second Bidirectional LSTM layer with reduced units and regularization
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    # Third Bidirectional LSTM layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    # Fourth Bidirectional LSTM layer with reduced units
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

     # Five Bidirectional LSTM layer with reduced units
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    # Final Bidirectional LSTM layer without return_sequences for the final output
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),

    # Output Dense layer with tanh activation
    tf.keras.layers.Dense(576, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001))
])


In [ ]:
# Compile model with the combined loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=lambda y_true, y_pred: combined_loss(y_true, y_pred, alpha=0.5, beta=0.1))
model.summary()

In [ ]:
# Callbacks for learning rate reduction and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
# Train model
history = model.fit(X_train_train, y_train_train,
                    epochs=300,
                    validation_data=(X_valid, y_valid),
                    batch_size=16,
                    callbacks=[reduce_lr, early_stopping])

In [ ]:
# Save the final model to a safe location
final_model_path = '/content/drive/MyDrive/Scaled_Test/final_best_model.h5'
model.save(final_model_path)
print(f"Best model saved to: {final_model_path}")

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss over epochs
def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.show()

plot_loss(history)


In [ ]:
input_test_data_list = []
input_file_names = []  # To store file names for saving predictions

def load_test_data_with_filenames(input_dir):
    input_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.csv')])

    for input_file in input_files:
        input_data = pd.read_csv(os.path.join(input_dir, input_file), header=None).iloc[:, 59:59+576].values.flatten()
        input_test_data_list.append(input_data)
        input_file_names.append(input_file)  # Save file names for later use

    return np.array(input_test_data_list), input_file_names

# Load test data
X_test, input_file_names = load_test_data_with_filenames(input_test_dir)
# Generate predictions
predictions_normalized = model.predict(X_test_normalized)

# Inverse transform predictions to original scale
predictions = output_scaler.inverse_transform(predictions_normalized.reshape(-1, 2)).reshape(predictions_normalized.shape)

# Save predictions with the same filenames
output_predictions_dir = '/content/drive/MyDrive/Scaled_Test/Predictions'

if not os.path.exists(output_predictions_dir):
    os.makedirs(output_predictions_dir)

for i, prediction in enumerate(predictions):
    output_file_path = os.path.join(output_predictions_dir, input_file_names[i])
    prediction_flat = prediction.flatten()  # Flatten prediction for saving
    pd.DataFrame(prediction_flat).to_csv(output_file_path, index=False, header=False)

print(f"Predictions saved in directory: {output_predictions_dir}")

In [ ]:
for i in [22, 26, 30, 34, 48]:
    for j in range(1, 41):
      file_path = f'/content/drive/MyDrive/Scaled_Test/Combined/predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.csv'
      data = pd.read_csv(file_path)

# Extract Prediction and Actual columns
      predictions = data['Prediction']
      actuals = data['Actual']

# Plot predictions and actuals
      plt.figure(figsize=(50, 30))
      plt.plot(predictions, label='Predictions', linestyle='-', marker='.')
      plt.plot(actuals, label='Actuals', linestyle='-', marker='.')
      plt.title(f'predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}')
      plt.xlabel('Index')
      plt.ylabel('Values (Normalized)')
      plt.legend()
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f'/content/drive/MyDrive/Scaled_Test/Plots/predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.png')
      plt.close()

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os

# Function to calculate Mean Squared Error (MSE)
def calculate_mse(predictions, actuals):
    """
    Calculates the Mean Squared Error (MSE) between predicted and actual values.
    """
    return mean_squared_error(actuals, predictions)

# Initialize lists to store results
results = []

# Min-Max scaler for normalization
scaler = MinMaxScaler()

# Output directory for combined files
combined_dir = '/content/drive/MyDrive/Scaled_Test/Combined'
os.makedirs(combined_dir, exist_ok=True)

# Iterate through each sample
for run in [22, 26, 30, 34, 48]:
    for j in range(1, 41):
        # Construct file paths
        pred_path = f'/content/drive/MyDrive/Scaled_Test/Predictions/Run_{run}_BL4_s39708_InjPulsePower_InjEnergyFactor_{j}.csv'
        actual_path = f'/content/drive/MyDrive/Scaled_Test/Output/Run_{run}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.csv'

        # Read the prediction and actual CSVs
        df_pred = pd.read_csv(pred_path, header=None).values.flatten()
        df_actual = pd.read_csv(actual_path, header=None).iloc[:, 59:59+576].values.flatten()

        # Create DataFrames for predicted and actual values
        df_preds = pd.DataFrame(df_pred, columns=['Prediction'])
        df_actuals = pd.DataFrame(df_actual, columns=['Actual'])

        # Calculate the total energy for predicted and actual values
        predicted_energy = df_preds['Prediction'].sum() * 0.025  # Sum and scale predictions
        ground_truth_energy = df_actuals['Actual'].sum() * 0.025  # Sum and scale normalized actuals

        # Calculate the energy difference
        energy_difference = abs(predicted_energy - ground_truth_energy)

        # Calculate MSE for the current prediction and actual values
        mse = calculate_mse(df_preds['Prediction'], df_actuals['Actual'])

        # Append the result for this file to the results list
        results.append({
            "Run": run,
            "File": j,
            "MSE": mse,
            "Predicted Energy": predicted_energy,
            "Ground Truth Energy": ground_truth_energy,
            "Energy Difference": energy_difference
        })

        # Combine predicted and normalized actual values into one DataFrame
        combined_df = pd.concat([df_preds, df_actuals], axis=1)

        # Save the combined DataFrame to a new CSV
        combined_file_path = f'/content/drive/MyDrive/Scaled_Test/Combined/predicted_Run_{run}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.csv'
        combined_df.to_csv(combined_file_path, index=False)

        print(f"Saved combined file: {combined_file_path}")

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_path = '/content/drive/MyDrive/Scaled_Test/energy_comparison_results.csv'
results_df.to_csv(results_path, index=False)

print(f"Energy comparison results saved to: {results_path}")


In [ ]:
results_df['Energy Difference'].mean()

In [ ]:
import gc  # For manual garbage collection
import pandas as pd
import matplotlib.pyplot as plt
for i in [22, 26, 30, 34, 48]:
    for j in range(1, 41):
        file_path = f'/content/drive/MyDrive/Scaled_Test/Combined/predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.csv'

        # Read only the necessary columns to save memory
        data = pd.read_csv(file_path, usecols=['Prediction', 'Actual'])

        # Extract Prediction and Actual columns
        predictions = data['Prediction']
        actuals = data['Actual']

        # Plot predictions and actuals
        plt.figure(figsize=(20, 12))  # Reduce figure size to conserve memory
        plt.plot(predictions, label='Predictions', linestyle='-', marker='.', markersize=2)
        plt.plot(actuals, label='Actuals', linestyle='-', marker='.', markersize=2)
        plt.title(f'predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}')
        plt.xlabel('Index')
        plt.ylabel('Values (Normalized)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        # Save the plot
        plot_path = f'/content/drive/MyDrive/Scaled_Test/Plots/predicted_Run_{i}_BL4_s39708_UVDBS_Power_InjEnergyFactor_{j}.png'
        plt.savefig(plot_path, dpi=300)  # Use higher DPI for better resolution
        plt.close()

        # Clear memory
        del data, predictions, actuals
        gc.collect()

print("All plots generated and saved successfully.")